<a href="https://colab.research.google.com/github/xhiroga/til/blob/main/software-engineering/haystack/quickstart/haystack_03_scalable_qa_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [Build a Scalable Question Answering System | Haystack](https://haystack.deepset.ai/tutorials/03_scalable_qa_system)

In [1]:
%%bash

nvidia-smi

Sun Sep 24 03:19:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    44W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch,inference]


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Obtaining dependency information for farm-haystack[colab,elasticsearch,inference,preprocessing] from https://files.pythonhosted.org/packages/31/db/e81141e15cecf1abc1238d9aee55f66310274878a034dd96603703620e9c/farm_haystack-1.20.1-py3-none-any.whl.metadata
  Obtaining dependency information for canals==0.7.0 from https://files.pythonhosted.org/packages/08/2f/62e9e455c4cf183ebf6672c47befabb6a36a37a8cdc86256a5632d6ace10/canals-0.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for events from https://files.pythonhosted.org/packages/25/ed/e47dec0626edd468c84c04d97769e7ab4ea6457b7f54dcb3f72b17fcd876/Events-0.5-py3-none-any.whl.metadata
  Obtaining dependency information for posthog from https://files.pythonhosted.org/packages/a7/73/35758818228c70348be4c3c6

In [3]:
from haystack.telemetry import tutorial_running

tutorial_running(3)


In [4]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)


## Initializing the ElasticsearchDocumentStore

In [5]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2


In [6]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch


In [7]:
import time

time.sleep(30)


In [8]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")


## Indexing Documents with a Pipeline

In [9]:
from haystack.utils import fetch_archive_from_http

doc_dir = "data/build_a_scalable_question_answering_system"

fetch_archive_from_http(
    url="https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip",
    output_dir=doc_dir,
)


INFO:haystack.utils.import_utils:Fetching from https://s3.eu-central-1.amazonaws.com/deepset.ai-farm-qa/datasets/documents/wiki_gameofthrones_txt3.zip to 'data/build_a_scalable_question_answering_system'


True

In [10]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [11]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


In [13]:
!ls data/build_a_scalable_question_answering_system | head

0_Game_of_Thrones__season_8_.txt
101_Titties_and_Dragons.txt
102_The_Princess_and_the_Queen.txt
10_Beyond_the_Wall__Game_of_Thrones_.txt
118_Dark_Wings__Dark_Words.txt
119_Walk_of_Punishment.txt
11_The_Dragon_and_the_Wolf.txt
120_And_Now_His_Watch_Is_Ended.txt
121_The_Bear_and_the_Maiden_Fair.txt
126_Kissed_by_Fire.txt


In [ ]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)


## Initializing the Retriever

In [15]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)


## Initializing the Reader

In [16]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)


INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.


INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


## Creating the Retriever-Reader Pipeline

In [17]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])


## Asking a Question

In [18]:
prediction = querying_pipeline.run(
    query="Who is the father of Arya Stark?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)


Inferencing Samples: 100%|██████████| 1/1 [00:03<00:00,  3.31s/ Batches]


In [19]:
from pprint import pprint

pprint(prediction)


{'answers': [<Answer {'answer': 'Eddard', 'type': 'extractive', 'score': 0.993372917175293, 'context': "s Nymeria after a legendary warrior queen. She travels with her father, Eddard, to King's Landing when he is made Hand of the King. Before she leaves,", 'offsets_in_document': [{'start': 207, 'end': 213}], 'offsets_in_context': [{'start': 72, 'end': 78}], 'document_ids': ['9e3c863097d66aeed9992e0b6bf1f2f4'], 'meta': {'_split_id': 4, '_split_overlap': [{'range': [0, 266], 'doc_id': '241c8775e39c6c937c67bbd10ccc471c'}, {'range': [960, 1200], 'doc_id': '87e8469dcf7354fd2a25fbd2ba07c543'}]}}>,
             <Answer {'answer': 'Ned', 'type': 'extractive', 'score': 0.9753613471984863, 'context': "k in the television series.\n\n====Season 1====\nArya accompanies her father Ned and her sister Sansa to King's Landing. Before their departure, Arya's h", 'offsets_in_document': [{'start': 630, 'end': 633}], 'offsets_in_context': [{'start': 74, 'end': 77}], 'document_ids': ['7d3360fa29130e69ea6b2b

In [20]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium` and `all`


'Query: Who is the father of Arya Stark?'
'Answers:'
[   {   'answer': 'Eddard',
        'context': 's Nymeria after a legendary warrior queen. She travels '
                   "with her father, Eddard, to King's Landing when he is made "
                   'Hand of the King. Before she leaves,'},
    {   'answer': 'Ned',
        'context': 'k in the television series.\n'
                   '\n'
                   '====Season 1====\n'
                   'Arya accompanies her father Ned and her sister Sansa to '
                   "King's Landing. Before their departure, Arya's h"},
    {   'answer': 'Lord Eddard Stark',
        'context': 'rk daughters.\n'
                   '\n'
                   'During the Tourney of the Hand to honour her father Lord '
                   'Eddard Stark, Sansa Stark is enchanted by the knights '
                   'performing in the event.'},
    {   'answer': 'Joffrey',
        'context': 'Mycah, sparring in the woods with broomsticks.  Arya '
    

In [26]:
pprint(querying_pipeline.run(
    query="Where does Arya Stark live in?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
))

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 12.45 Batches/s]

{'answers': [<Answer {'answer': 'Winterfell', 'type': 'extractive', 'score': 0.628581702709198, 'context': "Stark is the oldest son of Eddard and Catelyn Stark's and the heir to Winterfell. His direwolf is called Grey Wind. Robb becomes involved in the war a", 'offsets_in_document': [{'start': 689, 'end': 699}], 'offsets_in_context': [{'start': 70, 'end': 80}], 'document_ids': ['7ed7e4c40c15696ac06c27c8dcc77684'], 'meta': {'_split_id': 10, '_split_overlap': [{'range': [0, 175], 'doc_id': '2629f39cb972aea6477868dbcf9e79f6'}, {'range': [935, 1106], 'doc_id': 'a5fa1d00b634d3cdda5a8997d5020cb9'}]}}>,
             <Answer {'answer': 'In the Riverlands', 'type': 'extractive', 'score': 0.41520828008651733, 'context': '==Plot==\n\n===In the Riverlands===\nDisguised as Walder Frey, Arya Stark kills all the men of House Frey with poisoned wine, avenging the Red Wedding ("', 'offsets_in_document': [{'start': 13, 'end': 30}], 'offsets_in_context': [{'start': 13, 'end': 30}], 'document_ids': ['62d9